# Deploy your sentiment analysis model and predict

## Setup environment

In [51]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

role = sagemaker.get_execution_role()
model_artefact = 's3://sagemaker-us-east-1-175748383800/pytorch-training-2020-06-28-16-37-40-902/output/model.tar.gz'

## Create endpoint

In [52]:
class SentimentAnalysis(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session=sagemaker_session, serializer=json_serializer, 
                         deserializer=json_deserializer, content_type='application/json')

model = PyTorchModel(model_data=model_artefact,
                   name=name_from_base('roberta-model'),
                   role=role, 
                   entry_point='predictor.py',
                   source_dir='source_dir',
                   framework_version='1.5.0',
                   predictor_cls=SentimentAnalysis)

In [53]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

---------------!

## Predict

In [126]:
test_data = {"text": "Use your brain please for the sake of everyone around you"}
print(test_data)

{'text': 'Use your brain please for the sake of everyone around you'}


In [127]:
prediction = predictor.predict(test_data)

In [128]:
print(f'Review text: {test_data}')
print(f'Sentiment  : {prediction}')

Review text: {'text': 'Use your brain please for the sake of everyone around you'}
Sentiment  : neutral
